In [1]:
%pip install sktime==0.11.3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import timeit
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import fbeta_score, make_scorer
import tensorflow as tf
from pickle import dump

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_from_arff_to_dataframe as load_arff
#numpy.version.version

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [5]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
X, y = load_arff("AllCandidas_TRAIN_V2.arff")
print(np.shape(X))
print(type(X))
precision = []
accuracy = []
f1 = []
recall = []
def gpu(X, y, kf):
  with tf.device('/device:GPU:0'):
    i= 0
    for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)    
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #print(X_train.head())
        #print(type(X_train))
        print("Treining EE...")
        KNTSC = KNeighborsTimeSeriesClassifier()
        print(f"Interaction {i}")
        print("In trainning...")
        KNTSC.fit(X_train, y_train)
        print("In prediction...")
        i=i+1
        
        # Predict and print accuracy
        predictions = KNTSC.predict(X_test)
        precision.append(precision_score(y_test, predictions, average='macro',zero_division=1))
        accuracy.append(accuracy_score(y_test, predictions))
        recall.append(recall_score(y_test, predictions, average='macro',zero_division=1))
        f1.append(f1_score(y_test, predictions, average='macro'))
        #print("In prediction...")
        # Predict and print accuracy
       # predictions = HC2.predict(X_test)
    print(type(X_train))
    print(np.shape(X_train))
    print(f"Precision: {np.mean(precision)}, Accuracy:{np.mean(accuracy)}, Recall: {np.mean(recall)}, F1: {np.mean(f1)}")
    return np.mean(precision), np.mean(accuracy), np.mean(recall), np.mean(f1), np.std(accuracy), KNTSC

(12256, 1)
<class 'pandas.core.frame.DataFrame'>


In [6]:
if tf.config.list_physical_devices('GPU'):
  print("Using MirroredStrategy")
  strategy = tf.distribute.MirroredStrategy()
else:  # Use the Default Strategy
  print("Using Default Strategy")
  strategy = tf.distribute.get_strategy()

with strategy.scope():
  # Do something interesting
  print(tf.Variable(1.))
  precision, accuracy, recall, f1, std, KNTSC = gpu(X,y,kf)

Using MirroredStrategy
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
MirroredVariable:{
  0: <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
}
Treining EE...
Interaction 0
In trainning...
In prediction...
Treining EE...
Interaction 1
In trainning...
In prediction...
Treining EE...
Interaction 2
In trainning...
In prediction...
Treining EE...
Interaction 3
In trainning...
In prediction...
Treining EE...
Interaction 4
In trainning...
In prediction...
Treining EE...
Interaction 5
In trainning...
In prediction...
Treining EE...
Interaction 6
In trainning...
In prediction...
Treining EE...
Interaction 7
In trainning...
In prediction...
Treining EE...
Interaction 8
In trainning...
In prediction...
Treining EE...
Interaction 9
In trainning...
In prediction...
<class 'pandas.core.frame.DataFrame'>
(11031, 1)
Precision: 0.9702471818290999, Accuracy:0.9703832606452043, Recall: 0.9704152546747599, F1: 0.9702348385506031


In [7]:
df_metrics = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall','f1_score','std'])
df_metrics['precision'] = precision
df_metrics['accuracy'] = accuracy
df_metrics['recall'] = recall
df_metrics['f1_score'] = f1
df_metrics['std'] = std
df_metrics

C:\Users\mlb\AppData\Local\Temp/ipykernel_33368/3617186161.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_metrics = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0],


,precision,accuracy,recall,f1_score,std
0,0.970247,0.970383,0.970415,0.970235,0.006481


In [8]:
#save metrics
print("Saving metrics...")
df_metrics.to_csv("results/" + 'df_metrics_KNTSC.csv', index=False)
#save the model
print("Saving model...")
dump(KNTSC, open('results/'+'KNTSC_model.pkl', 'wb'))
print("Metrics of model:")
df_metrics

Saving metrics...
Saving model...
Metrics of model:


,precision,accuracy,recall,f1_score,std
0,0.970247,0.970383,0.970415,0.970235,0.006481


In [6]:
#if __name__ == "__main__":
    # Load data
def gpu():
  with tf.device('/device:GPU:0'):
    res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall','f1_score'])
    X_train, y_train = load_arff("AllCandidas_TRAIN_2.arff")
    X_test, y_test = load_arff("AllCandidas_TEST_2.arff")
    X_train.head()
    print(y_train[0:5])
    print(y_test[0:5])
    # Fit HC2
    print("Treining KNTSC...")
    KNTSC = KNeighborsTimeSeriesClassifier()
    KNTSC.fit(X_train, y_train)
    print("In prediction...")
    # Predict and print accuracy
    predictions = KNTSC.predict(X_test)
    res['precision'] = precision_score(y_test, predictions, average='macro')
    res['accuracy'] = accuracy_score(y_test, predictions)
    res['recall'] = recall_score(y_test, predictions, average='macro')
    res['f1_score'] = f1_score(y_test, predictions, average='macro')
    return res , KNTSC
df_metrics, KNTSC_model = gpu()

C:\Users\mlb\AppData\Local\Temp/ipykernel_20960/3223071326.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],


['6' '5' '1' '1' '5']
['6' '2' '3' '2' '1']
Treining KNTSC...
In prediction...


In [7]:
#save metrics
print("Saving metrics...")
df_metrics.to_csv('results/'+ 'df_metrics_KNTSC.csv', index=False)
#save the model
print("Saving model...")
dump(KNTSC_model, open('results/'+'KNTSC_model.pkl', 'wb'))
print("Metrics of model:")
df_metrics

Saving metrics...
Saving model...
Metrics of model:


,precision,accuracy,recall,f1_score
0,0.950262,0.952645,0.950735,0.950385


In [8]:
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)

GPU (s):


C:\Users\mlb\AppData\Local\Temp/ipykernel_20960/3223071326.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],


['6' '5' '1' '1' '5']
['6' '2' '3' '2' '1']
Treining KNTSC...
In prediction...


C:\Users\mlb\AppData\Local\Temp/ipykernel_20960/3223071326.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],


['6' '5' '1' '1' '5']
['6' '2' '3' '2' '1']
Treining KNTSC...
In prediction...


...

['6' '5' '1' '1' '5']
['6' '2' '3' '2' '1']
Treining KNTSC...
In prediction...


C:\Users\mlb\AppData\Local\Temp/ipykernel_20960/3223071326.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],


['6' '5' '1' '1' '5']
['6' '2' '3' '2' '1']
Treining KNTSC...
In prediction...


C:\Users\mlb\AppData\Local\Temp/ipykernel_20960/3223071326.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],


['6' '5' '1' '1' '5']
['6' '2' '3' '2' '1']
Treining KNTSC...
In prediction...
383.6802126
